# Setting-up environment

In [1]:
!cp -r ../input/updated-code-3-7/* /kaggle/working/

In [ ]:
!pip install -r requirements.txt

In [2]:
!pip install lightfm tqdm optuna ipykernel matplotlib implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for implicit from https://files.pythonhosted.org/packages/cd/cc/deac70cae8cc32c9885d0cd73bc66e1b3cbea36ae7080b8c83995eaf5322/implicit-0.7.2-cp310-cp310-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 58.9 MB/s eta 0:00:00:00:0100:01
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=464219 sha256=3ea656005763e80047107ec935aaa0b9ed2cc93c8d0ab9f0d45498797bfea1bb
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
#!python run_compile_all_cython.py

# Imports

In [3]:
import time
import optuna
import numpy as np
import pandas as pd
import scipy.sparse as sps
from scipy.sparse import vstack, csr_matrix
from scipy.stats import loguniform
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k
import matplotlib.pyplot as plt
from multiprocessing import cpu_count, Lock

In [4]:
from Recommenders.BaseRecommender import BaseRecommender
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.NonPersonalizedRecommender import TopPop

#---- CF
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

#---- Matrix Factorization
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#---- CF w/ ML
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from implicit.als import AlternatingLeastSquares

#---- Others
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.Recommender_utils import check_matrix

In [5]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout

In [6]:
seed = 69
np.random.seed(seed)

# Data Import and Preprocessing

In [7]:
# Import training data
URM_path = "../input/data-books/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                header=0,
                                dtype={0:int, 1:int, 2:int},
                                engine='python')

URM_all_dataframe.columns = ["user_id", "item_id", "interaction"]

In [8]:
# Import target users
target_path = "../input/data-books/data_target_users_test.csv"
target_dataframe= pd.read_csv(filepath_or_buffer=target_path,
                                header=0,
                                dtype={0:int},
                                engine='python')
target_dataframe.columns = ["user_id"]
target_dataframe

,user_id
0,1
1,2
2,3
3,4
4,5
...,...
10877,13020
10878,13021
10879,13022
10880,13023


In [9]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    ratings = pd.merge(left=ratings,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    ratings = pd.merge(left=ratings,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return ratings

In [10]:
# Call preprocess data function
ratings = preprocess_data(URM_all_dataframe)

12638 1 13024
22222 1 22347


## Conversion to Sparse Matrix

In [11]:
URM = sps.coo_matrix((ratings.interaction.values, (ratings.mapped_user_id.values, ratings.mapped_item_id.values)))

## Data Splits

In [12]:
#urm_train, urm_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
#evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2127 (16.8%) Users that have less than 1 test interactions


# **Custom Models**

In [13]:
class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2


    def fit(self, alpha=0.5):
        self.alpha = alpha

    def _compute_item_score(self, user_id_array, items_to_compute):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [14]:
class LightFMCFRecommender(BaseRecommender):
    """LightFMCFRecommender"""

    RECOMMENDER_NAME = "LightFMCFRecommender"

    def __init__(self, URM_train):
        super(LightFMCFRecommender, self).__init__(URM_train)
    
    
    def fit(self, epochs = 300, user_alpha=1e-6, item_alpha = 1e-6, n_factors = 10, n_threads = 4, max_sampled=3, loss='warp', learning_schedule='adagrad'):
        
        # Let's fit a WARP model
        self.lightFM_model = LightFM(loss=loss,
                                     user_alpha=user_alpha,
                                     item_alpha=item_alpha,
                                     no_components=n_factors,
                                     max_sampled=max_sampled,
                                     learning_schedule=learning_schedule)

        self.lightFM_model = self.lightFM_model.fit(self.URM_train, 
                                       epochs=epochs,
                                       num_threads=n_threads,
                                       verbose=True)


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        # Create a single (n_items, ) array with the item score, then copy it for every user
        items_to_compute = np.arange(self.n_items)
        
        item_scores = - np.ones((len(user_id_array), self.n_items)) * np.inf

        for user_index, user_id in enumerate(user_id_array):
            item_scores[user_index] = self.lightFM_model.predict(int(user_id), 
                                                                 items_to_compute)

        return item_scores

In [15]:
class GeneralizedLinearHybridRecommender(BaseRecommender):
    """
    This recommender merges N recommendes by weighting their ratings
    """

    RECOMMENDER_NAME = "GeneralizedLinearHybridRecommender"

    def __init__(self, URM_train, recommenders: list, verbose=True):
        self.RECOMMENDER_NAME = ''
        for recommender in recommenders:
            self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + recommender.RECOMMENDER_NAME[:-11]
        self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + 'HybridRecommender'

        super(GeneralizedLinearHybridRecommender, self).__init__(URM_train, verbose=verbose)

        self.recommenders = recommenders

    def fit(self, alphas=None):
        self.alphas = alphas

    def save_model(self, folder_path, file_name=None):
        pass

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        result = self.alphas[0]*self.recommenders[0]._compute_item_score(user_id_array,items_to_compute)
        for index in range(1,len(self.alphas)):
            result = result + self.alphas[index]*self.recommenders[index]._compute_item_score(user_id_array,items_to_compute)
        return result

In [16]:
class FastIALSRecommender(BaseMatrixFactorizationRecommender):
    RECOMMENDER_NAME = "FastIALSRecommender"

    AVAILABLE_CONFIDENCE_SCALING = ["linear", "log"]
    
    def __init__(self, URM_train, verbose=True):
        super().__init__(URM_train, verbose=verbose)
        
    def fit(self,
            factors=20,
            regularization=1e-3,
            iterations=100,
            calculate_training_loss=False,
            num_threads=0,
            confidence_scaling='linear',
            alpha=1.0,
            epsilon=0,
            #---- Do not change
            use_native=True,
            use_cg=True,
            use_gpu=True):
        if confidence_scaling not in self.AVAILABLE_CONFIDENCE_SCALING:
           raise ValueError("Value for 'confidence_scaling' not recognized. Acceptable values are {}, provided was '{}'".format(self.AVAILABLE_CONFIDENCE_SCALING, confidence_scaling))

        self.alpha = alpha
        self.epsilon = epsilon
        self.num_factors = factors
        self.reg = regularization
        
        self.USER_factors = self._init_factors(self.n_users, False)  # don't need values, will compute them
        self.ITEM_factors = self._init_factors(self.n_items)
        
        self.recommender = AlternatingLeastSquares(factors=factors, regularization=regularization,
                                                        use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
                                                        iterations=iterations,
                                                        calculate_training_loss=calculate_training_loss,
                                                        num_threads=num_threads)
        
        self._build_confidence_matrix(confidence_scaling)
        self.recommender.fit(self.C, show_progress=self.verbose)
        
        self.USER_factors = self.recommender.user_factors.to_numpy()
        self.ITEM_factors = self.recommender.item_factors.to_numpy()
        
    
    def _linear_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha*C.data

        return C

    def _log_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha * np.log(1.0 + C.data / self.epsilon)

        return C
    
    def _build_confidence_matrix(self, confidence_scaling):

        if confidence_scaling == 'linear':
            self.C = self._linear_scaling_confidence()
        else:
            self.C = self._log_scaling_confidence()

        self.C_csc= check_matrix(self.C.copy(), format="csc", dtype = np.float32)
    
    def _init_factors(self, num_factors, assign_values=True):

        if assign_values:
            return self.num_factors**-0.5*np.random.random_sample((num_factors, self.num_factors))

        else:
            return np.empty((num_factors, self.num_factors))

# **Best Model Params**

In [17]:
ItemKNN_params = {
    'topK': 6,
    'shrink': 15,
    'similarity': 'jaccard',
    'normalize': False
}

In [18]:
RP3beta_params = {
    'alpha': 0.307953246083667, 
    'beta': 0.3073797221110665, 
    'topK': 59, 
    'normalize_similarity': True
}

In [19]:
alpha_itemknn_rp3beta=0.7381515719042592

In [20]:
UserKNN_params = {
    'topK': 470,
    'shrink': 0,
    'similarity': 'cosine',
    'normalize': True
}

In [21]:
LightFM_params = {
                  'n_factors': 482,
                  'max_sampled': 5,
                  'user_alpha': 0.00023989649900734266,
                  'item_alpha': 9.740651135253414e-05
                 }

In [22]:
SLIM_params = {
    'l1_ratio': 0.013752256221164005,
    'alpha': 0.0031943927190071775,
    'topK': 622
}

In [23]:
NMF_params = {
    'l1_ratio': 0.005734775635120469,
    'num_factors': 134,
    'beta_loss': 'frobenius',
    'init_type': 'nndsvda',
    'solver': 'multiplicative_update'
}

In [24]:
IALS_params = {
    'confidence_scaling': 'log',
    'epsilon': 0.11624415533664904,
    'factors': 116,
    'regularization': 0.005454427904241962,
    'alpha': 1.7221339971074425
}

# Tuning SlimEN

In [25]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame(columns=["result"])

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        # Create a DataFrame from the current trial's results
        trial_df = pd.DataFrame([hyperparam_dict])

        # Use concat instead of append
        self.results_df = pd.concat([self.results_df, trial_df], ignore_index=True)

In [26]:
def objective_function_SLIM(optuna_trial):

    recommender_instance = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train)
    recommender_instance.fit(
                             l1_ratio = optuna_trial.suggest_float("l1_ratio", 1e-7, 0.1),
                             alpha = optuna_trial.suggest_float("alpha", 0, 1),
                             topK = optuna_trial.suggest_int("topK", 5, 2500),
                             workers = int(cpu_count())
                             #positive_only = optuna_trial.suggest_categorical("positive_only", [True, False]),
                            )

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["RECALL"]

In [27]:
optuna_study_SLIM = optuna.create_study(direction="maximize")

save_results_SLIM = SaveResults()

optuna_study_SLIM.optimize(objective_function_SLIM,
                      callbacks=[save_results_SLIM],
                      n_trials = 1000)

[I 2023-12-20 20:27:42,693] A new study created in memory with name: no-name-9720942a-ac53-4fff-94c5-b4dde0662202


SLIMElasticNetRecommender: URM Detected 240 ( 1.9%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 123 ( 0.6%) items with no interactions.


100%|█████████▉| 22216/22222 [08:03<00:00, 45.91it/s] 


EvaluatorHoldout: Processed 10511 (100.0%) in 9.41 sec. Users per second: 1117


[I 2023-12-20 20:35:56,214] Trial 0 finished with value: 0.012246115195107047 and parameters: {'l1_ratio': 0.03058569985023565, 'alpha': 0.6514852683204296, 'topK': 1820}. Best is trial 0 with value: 0.012246115195107047.


...

EvaluatorHoldout: Processed 10511 (100.0%) in 11.98 sec. Users per second: 878


[I 2023-12-21 00:46:50,283] Trial 20 finished with value: 0.11998517618481742 and parameters: {'l1_ratio': 0.010980473971047941, 'alpha': 0.03443415298193212, 'topK': 267}. Best is trial 16 with value: 0.13921079642329776.


SLIMElasticNetRecommender: URM Detected 240 ( 1.9%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 123 ( 0.6%) items with no interactions.


  0%|          | 0/22222 [00:00<?, ?it/s][W 2023-12-21 00:46:56,221] Trial 21 failed with parameters: {'l1_ratio': 0.0014372648241861802, 'alpha': 0.0031038317509641832, 'topK': 673} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 856, in next
    item = self._items.popleft()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_47/547765023.py", line 4, in objective_function_SLIM
    recommender_instance.fit(
  File "/kaggle/working/Recommenders/SLIM/SLIMElasticNetRecommender.py", line 242, in fit
    for values_, rows_, cols_ in pool.imap_unordered(_pfit, itemchunks, pool_chunksize):
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py"

KeyboardInterrupt: 

In [28]:
#BEST RECALL
#Trial 16 finished with value: 0.13921079642329776 and parameters: {'l1_ratio': 0.01998720297988901, 'alpha': 0.0032805805487669745, 'topK': 698}. Best is trial 16 with value: 0.13921079642329776.

In [ ]:
#{'l1_ratio': 0.003120566520870887, 'alpha': 0.07957855829190108, 'topK': 1478}. Best is trial 8 with value: 0.11397645151525244

In [ ]:
pruned_trials = [t for t in optuna_study_SLIM.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in optuna_study_SLIM.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(optuna_study_SLIM.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
print("  Value Validation: ", optuna_study_SLIM.best_trial.value)

print("Best params:")
print(optuna_study_SLIM.best_trial.params)